# rgb-lib issuance and transfer demo
Example of asset issuance and transfer using [rgb-lib](https://github.com/RGB-Tools/rgb-lib) via its [Python bindings](https://github.com/RGB-Tools/rgb-lib-python)

### imports

In [ ]:
import os
import shutil

import magic
import matplotlib.pyplot as plot
import qrcode
from IPython.display import Image, display

import rgb_lib

### common values

In [ ]:
electrum_url = 'tcp://electrs:50001'
fee_rate = 2
transport_endpoints = ['rpc://proxy:3000/json-rpc']
bitcoin_network = rgb_lib.BitcoinNetwork.REGTEST
supported_schemas = [rgb_lib.AssetSchema.NIA, rgb_lib.AssetSchema.CFA]

---
# receiver
---

### generate a new wallet

In [ ]:
recv_keys = rgb_lib.generate_keys(bitcoin_network)
print('receiver wallet keys:')
print(' - mnemonic:', recv_keys.mnemonic)
print(' - master fingerprint:', recv_keys.master_fingerprint)
print(' - vanilla account xpub:', recv_keys.account_xpub_vanilla)
print(' - colored account xpub:', recv_keys.account_xpub_colored)

### initialize the wallet

In [ ]:
recv_data_dir = './data/recv_wallet'
recv_mnemonic = recv_keys.mnemonic
recv_fingerprint = recv_keys.master_fingerprint
recv_acc_xpub_van = recv_keys.account_xpub_vanilla
recv_acc_xpub_col = recv_keys.account_xpub_colored

if not os.path.exists(recv_data_dir):
    os.makedirs(recv_data_dir)

recv_wallet_data = rgb_lib.WalletData(
        data_dir=recv_data_dir,
        bitcoin_network=bitcoin_network,
        database_type=rgb_lib.DatabaseType.SQLITE,
        max_allocations_per_utxo=1,
        account_xpub_vanilla=recv_acc_xpub_van,
        account_xpub_colored=recv_acc_xpub_col,
        mnemonic=recv_mnemonic,
        master_fingerprint=recv_fingerprint,
        vanilla_keychain=None,
        supported_schemas=supported_schemas,
)

recv_wallet = rgb_lib.Wallet(recv_wallet_data)
recv_online = recv_wallet.go_online(False, electrum_url)

### get a Bitcoin address to add some funds
fund the address with some regtest bitcoins
```shell
./services.sh fund <address>
```

In [ ]:
recv_address = recv_wallet.get_address()

recv_image = qrcode.make(recv_address)
plot.imshow(recv_image,cmap='gray')
print(recv_address)

### create UTXOs to hold RGB allocations

In [ ]:
recv_wallet.create_utxos(recv_online, True, 5, None, fee_rate, False)

### check wallet unspents

In [ ]:
recv_unspents = recv_wallet.list_unspents(recv_online, settled_only=False, skip_sync=False)
for unspent in recv_unspents:
    print(unspent.utxo)
    for allocation in unspent.rgb_allocations:
        print(f'\t- {allocation}')

### prepare invoices to receive assets

In [ ]:
recv_data_1 = recv_wallet.blind_receive(None, rgb_lib.Assignment.ANY(), None, transport_endpoints, 1)
recv_data_2 = recv_wallet.witness_receive(None, rgb_lib.Assignment.ANY(), None, transport_endpoints, 1)
print(f'recipient ID 1 (blinded): {recv_data_1.recipient_id}')
print(f'recipient ID 2 (witness): {recv_data_2.recipient_id}')

---
# sender
---

### generate a new wallet
each time `rgb_lib.generate_keys()` is called a new mnemonic and xpub are created, giving a brand new wallet

In [ ]:
send_keys = rgb_lib.generate_keys(bitcoin_network)
print('new wallet keys:')
print(' - mnemonic:', send_keys.mnemonic)
print(' - master fingerprint:', send_keys.master_fingerprint)
print(' - vanilla account xpub:', send_keys.account_xpub_vanilla)
print(' - colored account xpub:', send_keys.account_xpub_colored)

### initialize the wallet
copy the generated mnemonic and xpub and set them to their respective variables below,
this way the bitcoin wallet won't change unexpectedly between runs

In [ ]:
send_data_dir = './data/send_wallet'
send_mnemonic = send_keys.mnemonic
send_fingerprint = send_keys.master_fingerprint
send_acc_xpub_van = send_keys.account_xpub_vanilla
send_acc_xpub_col = send_keys.account_xpub_colored

if not os.path.exists(send_data_dir):
    os.makedirs(send_data_dir)

send_wallet_data = rgb_lib.WalletData(
        data_dir=send_data_dir,
        bitcoin_network=bitcoin_network,
        database_type=rgb_lib.DatabaseType.SQLITE,
        max_allocations_per_utxo=1,
        account_xpub_vanilla=send_acc_xpub_van,
        account_xpub_colored=send_acc_xpub_col,
        mnemonic=send_mnemonic,
        master_fingerprint=send_fingerprint,
        vanilla_keychain=None,
        supported_schemas=supported_schemas,
)

send_wallet = rgb_lib.Wallet(send_wallet_data)
send_online = send_wallet.go_online(False, electrum_url)

### get a Bitcoin address to add some funds
fund the address with some regtest bitcoins
```shell
./services.sh fund <address>
```

In [ ]:
send_address = send_wallet.get_address()
print(send_address)

send_image = qrcode.make(send_address)
plot.imshow(send_image,cmap='gray')

### create UTXOs to hold RGB allocations

In [ ]:
send_wallet.create_utxos(send_online, True, 5, None, fee_rate, False)

### check wallet unspents

In [ ]:
send_unspents = send_wallet.list_unspents(send_online, settled_only=False, skip_sync=False)
for unspent in send_unspents:
    print(unspent.utxo)
    for allocation in unspent.rgb_allocations:
        print(f'\t- {allocation}')

### issue a NIA asset

In [ ]:
# asset data
name = 'Jupyter Fungible Token'
precision = 0
amounts = [1000]
ticker = 'JFT'

# issue the asset
nia_asset = send_wallet.issue_asset_nia(ticker, name, precision, amounts)
print(f'issued asset with ID: {nia_asset.asset_id}')

### issue a CFA asset
place an PNG image in the demo directory, named `sample.png`

In [ ]:
# asset data
name = 'JupyNFT'
precision = 0
amounts = [1]
description = 'Jupyter Collectible'
parent_id = None
file_path = 'sample.png'

if not os.path.exists(file_path):
    raise RuntimeError(f'missing image file: {file_path}')

# issue the asset
cfa_asset = send_wallet.issue_asset_cfa(name, description, precision, amounts, file_path)
print(f'issued asset with ID: {cfa_asset.asset_id}')

### list wallet assets

In [ ]:
assets = send_wallet.list_assets(filter_asset_schemas=[])

print('NIA assets:')
for asset in assets.nia:
    print(f'- {asset}')
    
print('\nCFA assets:')
for asset in assets.cfa:
    print(f'- {asset}')

### send some assets to the receiver's blinded UTXO
- create PSBT and RGB transfer
- anchor RGB transfer to PSBT
- post consignment to proxy server

#### NIA

In [ ]:
amount_nia = 10

recipient_map_nia = {
    nia_asset.asset_id: [
        rgb_lib.Recipient(
            recipient_id=recv_data_1.recipient_id,
            witness_data=None,
            assignment=rgb_lib.Assignment.FUNGIBLE(amount_nia),
            transport_endpoints=transport_endpoints,
        ),
    ]
}
txid = send_wallet.send(send_online, recipient_map_nia, False, fee_rate, 1, False)

print(f'NIA txid: {txid}')

#### CFA

In [ ]:
amount_cfa = 1
witness_data=rgb_lib.WitnessData(
    amount_sat=1000,
    blinding=None,
)

recipient_map_cfa = {
    cfa_asset.asset_id: [
        rgb_lib.Recipient(
            recipient_id=recv_data_2.recipient_id,
            witness_data=witness_data,
            assignment=rgb_lib.Assignment.FUNGIBLE(amount_cfa),
            transport_endpoints=transport_endpoints,
        ),
    ]
}
txid = send_wallet.send(send_online, recipient_map_cfa, False, fee_rate, 1, False)

print(f'CFA txid: {txid}')

### list asset transfers

In [ ]:
print('NIA asset transfers:')
send_transfers = send_wallet.list_transfers(nia_asset.asset_id)
for transfer in send_transfers:
    print(f'- {transfer}')
    
print('\nCFA asset transfers:')
send_transfers = send_wallet.list_transfers(cfa_asset.asset_id)
for transfer in send_transfers:
    print(f'- {transfer}')

---
# receiver
---

### refresh transfers
- check pending transfers for incoming consignments
- get the consignment from the proxy server and validate it
- ACK the transfer on the proxy server

In [ ]:
refreshed_transfers = recv_wallet.refresh(recv_online, None, [], False)
for key, val in refreshed_transfers.items():
    print(f'- {key}: {val}')

### list asset transfers

In [ ]:
print('NIA asset transfers:')
recv_transfers = recv_wallet.list_transfers(nia_asset.asset_id)
for transfer in recv_transfers:
    print(f'- {transfer}')

print('\nCFA asset transfers:')
recv_transfers = recv_wallet.list_transfers(cfa_asset.asset_id)
for transfer in recv_transfers:
    print(f'- {transfer}')

---
# sender
---

### refresh transfers
- check pending transfers for ACKs
- broadcast transaction for ACKed transfers

In [ ]:
refreshed_transfers = send_wallet.refresh(send_online, None, [], False)
for key, val in refreshed_transfers.items():
    print(f'- {key}: {val}')

### list asset transfers

In [ ]:
print('NIA asset transfers:')
send_transfers = send_wallet.list_transfers(nia_asset.asset_id)
for transfer in send_transfers:
    print(f'- {transfer}')
    
print('\nCFA asset transfers:')
send_transfers = send_wallet.list_transfers(cfa_asset.asset_id)
for transfer in send_transfers:
    print(f'- {transfer}')

---
### transaction confirmation
for the transfer to progress to a final state, mine a block
```shell
./services.sh mine
```

---
# receiver
---

### refresh transfers
- check pending transfers for transaction confirmations

In [ ]:
refreshed_transfers = recv_wallet.refresh(recv_online, None, [], False)
for key, val in refreshed_transfers.items():
    print(f'- {key}: {val}')

### list asset transfers

In [ ]:
print('NIA asset transfers:')
recv_transfers = recv_wallet.list_transfers(nia_asset.asset_id)
for transfer in recv_transfers:
    print(f'- {transfer}')
    assert(transfer.status == rgb_lib.TransferStatus.SETTLED)


print('\nCFA asset transfers:')
recv_transfers = recv_wallet.list_transfers(cfa_asset.asset_id)
for transfer in recv_transfers:
    print(f'- {transfer}')
    assert(transfer.status == rgb_lib.TransferStatus.SETTLED)

In [ ]:
recv_assets = recv_wallet.list_assets([])

print('NIA assets:')
for asset in recv_assets.nia:
    print(f'- {asset}')

print('\nCFA assets:')
magic_mime = magic.Magic(mime=True)
local_file_path = 'data/image.png'
for asset in recv_assets.cfa:
    print(f'- {asset}')
    media_mime = asset.media.mime
    media_path = asset.media.file_path
    file_mime = magic_mime.from_file(media_path)
    assert(media_mime == 'image/png')
    assert(file_mime == 'image/png')
    shutil.copyfile(media_path, local_file_path) 
    display(Image(filename=local_file_path))

---
# sender
---

### refresh transfers
- check pending transfers for transaction confirmations

In [ ]:
refreshed_transfers = send_wallet.refresh(send_online, None, [], False)
for key, val in refreshed_transfers.items():
    print(f'- {key}: {val}')

---
---
# backup + restore

### backup receiver wallet

In [ ]:
backup_path = './recv_wallet.backup'
backup_pass = 'password'
recv_wallet.backup(backup_path, backup_pass)
backup_size = os.path.getsize(backup_path)
print(f'backup complete ({backup_size} bytes)')

### restore wallet

In [ ]:
restore_data_dir = './data/restored_wallet'
rgb_lib.restore_backup(backup_path, backup_pass, restore_data_dir)
print('restore complete')

restored_wallet_data = rgb_lib.WalletData(
        data_dir=restore_data_dir,
        bitcoin_network=bitcoin_network,
        database_type=rgb_lib.DatabaseType.SQLITE,
        max_allocations_per_utxo=1,
        account_xpub_vanilla=recv_acc_xpub_van,
        account_xpub_colored=recv_acc_xpub_col,
        mnemonic=recv_mnemonic,
        master_fingerprint=recv_fingerprint,
        vanilla_keychain=None,
        supported_schemas=supported_schemas,
)

restored_wallet = rgb_lib.Wallet(restored_wallet_data)
restored_online = restored_wallet.go_online(False, electrum_url)
print('restored wallet instantiated\n')

In [ ]:
print('unspents:')
restored_unspents = restored_wallet.list_unspents(restored_online, settled_only=False, skip_sync=False)
for unspent in restored_unspents:
    print(unspent.utxo)
    for allocation in unspent.rgb_allocations:
        print(f'\t- {allocation}')

print('NIA asset transfers:')
restored_transfers = restored_wallet.list_transfers(nia_asset.asset_id)
for transfer in restored_transfers:
    print(f'- {transfer}')

print('\nCFA asset transfers:')
restored_transfers = restored_wallet.list_transfers(cfa_asset.asset_id)
for transfer in restored_transfers:
    print(f'- {transfer}')

restored_assets = restored_wallet.list_assets([])

print('NIA assets:')
for asset in restored_assets.nia:
    print(f'- {asset}')

print('\nCFA assets:')
magic_mime = magic.Magic(mime=True)
local_file_path = 'data/restored_image.png'
for asset in restored_assets.cfa:
    print(f'- {asset}')
    media_mime = asset.media.mime
    media_path = asset.media.file_path
    file_mime = magic_mime.from_file(media_path)
    assert(media_mime == 'image/png')
    assert(file_mime == 'image/png')
    shutil.copyfile(media_path, local_file_path) 
    display(Image(filename=local_file_path))